In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI SDK: 用于批量预测的自定义训练图像分类模型，具有可解释性

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/explainable_ai/sdk_custom_image_classification_batch_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fexplainable_ai%2Fsdk_custom_image_classification_batch_explain.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab Enterprise中打开
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/explainable_ai/sdk_custom_image_classification_batch_explain.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在Workbench中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/explainable_ai/sdk_custom_image_classification_batch_explain.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看
    </a>
  </td>
</table>

## 概述

本教程演示了如何使用Vertex AI SDK来训练和部署用于批量预测的自定义图像分类模型，并提供解释。

了解更多关于[Vertex Explainable AI](https://cloud.google.com/vertex-ai/docs/explainable-ai/overview)和[Vertex AI Batch Prediction](https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/get-batch-predictions)。

### 目标

在本教程中，您将学习如何使用 `Vertex AI Training and Explainable AI` 创建具有解释功能的自定义图像分类模型，然后学习如何使用 `Vertex AI Batch Prediction` 发出带解释的批量预测请求。您也可以使用 `gcloud` 命令行工具或在线使用 Cloud Console 创建自定义模型。

本教程使用以下谷歌云 ML 服务：

- `Vertex AI Training`
- `Vertex AI Batch Prediction`
- `Vertex Explainable AI`
- `Vertex AI Models`

执行的步骤包括：

- 创建用于训练 TensorFlow 模型的 `Vertex AI` 自定义作业。
- 查看经过训练模型的模型评估。
- 设置模型部署时的解释参数。
- 将经过训练的模型工件和解释参数上传为 `Model` 资源。
- 进行带解释的批量预测。

数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/overview)的[CIFAR10数据集](https://www.tensorflow.org/datasets/catalog/cifar10)。在本教程中使用的数据集版本已经集成到TensorFlow中。训练得到的模型可以预测输入图像属于10个类别中的哪一类，具体包括飞机、汽车、鸟、猫、鹿、狗、青蛙、马、船和卡车。

费用

本教程使用Google Cloud的计费组件：

* Vertex AI
* Cloud Storage

了解[Vertex AI的定价](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage的定价](https://cloud.google.com/storage/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

开始吧

### 为Python安装Vertex AI SDK和其他必需的软件包

In [ ]:
import os

! pip3 install --upgrade --quiet google-cloud-aiplatform
! pip3 install --upgrade --quiet google-cloud-storage
! pip3 install --upgrade --quiet tensorflow==2.15.1
! pip3 install --upgrade --quiet opencv-python-headless

if os.getenv("IS_TESTING"):
    ! apt-get update && apt-get install -y python3-opencv-headless
    ! apt-get install -y libgl1-mesa-dev

### 重新启动运行时（仅限Colab）

要使用新安装的包，您必须重新启动Google Colab上的运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 内核将重新启动。请等待其完成后再继续下一步。⚠️</b>
</div>

### 在Colab上验证您的笔记本环境

在Google Colab上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置Google Cloud项目信息并为Python初始化Vertex AI SDK

要开始使用Vertex AI，您必须拥有一个现有的Google Cloud项目并[启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

创建一个云存储桶

创建一个存储桶来存储中间工件，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶尚不存在：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $LOCATION -p $PROJECT_ID $BUCKET_URI

初始化Python的Vertex AI SDK

为您的项目和相应的存储桶初始化Python的Vertex AI SDK。

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持GPU的容器映像和分配给虚拟机（VM）实例的GPU数量。例如，要使用一个支持GPU的容器映像，并为每个VM分配4个Nvidia Telsa K80 GPU，您可以指定：

(aip.gapic.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，指定`(None, None)`以在CPU上运行容器映像。

了解有关您位置的[硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)

*注意*: 在GPU支持方面，TF 2.3 之前的版本在此教程中无法加载自定义模型。这是一个已知问题，在TF 2.3 中已修复 -- 这是由服务函数中生成的静态图操作引起的。如果在您自己的自定义模型上遇到此问题，请使用支持GPU的 TF 2.3 容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (None, None)

if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

为训练和预测设置预构建的 Docker 容器镜像。

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-11"

if TF[0] == "2":
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if TRAIN_GPU:
        TRAIN_VERSION = "tf-gpu.{}".format(TF)
    else:
        TRAIN_VERSION = "tf-cpu.{}".format(TF)
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)


TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    LOCATION.split("-")[0], TRAIN_VERSION
)
DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    LOCATION.split("-")[0], DEPLOY_VERSION
)

print("Training:", TRAIN_IMAGE, TRAIN_GPU, TRAIN_NGPU)
print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量 `TRAIN_COMPUTE` 和 `DEPLOY_COMPUTE` 来配置您将用于训练和预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`: 每个vCPU 3.75GB内存。
     - `n1-highmem`: 每个vCPU 6.5GB内存
     - `n1-highcpu`: 每个vCPU 0.9GB内存
 - `vCPUs`: 数量为\[2, 4, 8, 16, 32, 64, 96\]

*注意：以下情况不支持进行训练：*

 - `standard`: 2个vCPUs
 - `highcpu`: 2、4和8个vCPUs

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在你已经准备好开始创建自己的定制模型并为CIFAR10进行训练。

### 检查培训包

#### 包布局

在开始培训之前，您要查看如何为自定义培训工作组装Python包。解压后，包含以下目录/文件布局的包。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件 `setup.cfg` 和 `setup.py` 是将包安装到Docker镜像的操作环境中的说明。

文件 `trainer/task.py` 是执行自定义培训工作的Python脚本。*注意*，当在工作组规范中引用 `trainer/task.py` 时，目录斜杠会被替换为点，文件后缀（`.py`）会被删除（`trainer.task`）。

#### 包组装

在以下单元格中，您将创建培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: CIFAR10 image classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py的内容

在下一个单元格中，提供训练脚本task.py文件的内容。您不必详细说明。这只是供您参考浏览的。总结如下：

- 从命令行(`--model_dir`)获取保存模型结果的目录，如果未指定，则从环境变量`AIP_MODEL_DIR`获取。
- 从TF数据集(tfds)加载CIFAR10数据集。
- 使用TF.Keras模型API构建模型。
- 编译模型(`compile()`)。
- 根据参数`args.distribute`设置训练分布策略。
- 根据参数`args.epochs`和`args.steps`训练模型(`fit()`)。
- 将训练好的模型(`save(args.model_dir)`)保存到指定的模型目录。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for CIFAR-10

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv("AIP_MODEL_DIR"), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.01, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=10, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=200, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))
print('DEVICES', device_lib.list_local_devices())

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))

# Preparing dataset
BUFFER_SIZE = 10000
BATCH_SIZE = 64


def make_datasets_unbatched():

  # Scaling CIFAR10 data from (0, 255] to (0., 1.]
  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.0
    return image, label


  datasets, info = tfds.load(name='cifar10',
                            with_info=True,
                            as_supervised=True)
  return datasets['train'].map(scale).cache().shuffle(BUFFER_SIZE).repeat()


# Build the Keras model
def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      optimizer=tf.keras.optimizers.SGD(learning_rate=args.lr),
      metrics=['accuracy'])
  return model


# Train the model
NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS
train_dataset = make_datasets_unbatched().batch(GLOBAL_BATCH_SIZE)

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_cnn_model()

model.fit(x=train_dataset, epochs=args.epochs, steps_per_epoch=args.steps)
model.save(args.model_dir)

把培训脚本存储在您的云存储桶中

然后，您将培训文件夹打包成一个压缩的tar文件，并将其存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_cifar10.tar.gz

### 创建和运行自定义训练任务

要训练一个自定义模型，您需要执行两个步骤：

1. 创建一个自定义训练任务
2. 运行这个任务

#### 创建自定义训练任务

使用`CustomTrainingJob`类创建一个自定义训练任务，需要以下参数：

- `display_name`：自定义训练任务的可读名称。
- `container_uri`：训练容器镜像。
- `requirements`：训练容器镜像的软件包要求（例如 pandas）。
- `script_path`：训练脚本的相对路径。

In [ ]:
job = aiplatform.CustomTrainingJob(
    display_name="cifar10",
    script_path="custom/trainer/task.py",
    container_uri=TRAIN_IMAGE,
    requirements=["gcsfs==0.7.1", "tensorflow-datasets==4.4"],
)

print(job)

### 准备您的命令行参数

现在为您的自定义训练容器定义命令行参数：

- `args`：要传递给设定为容器入口点的可执行文件的命令行参数。
  - `--model-dir`：对于这些演示，使用这个命令行参数来指定存储模型工件的位置。
      - 直接：您将云存储位置作为命令行参数传递给您的训练脚本（设置变量`DIRECT = True`），或
      - 间接：服务将云存储位置作为环境变量`AIP_MODEL_DIR`传递给您的训练脚本（设置变量`DIRECT = False`）。在这种情况下，您需要在作业规范中告诉服务模型工件位置。
  - `"--epochs=" + EPOCHS`：训练的周期数。
  - `"--steps=" + STEPS`：每个周期的步数。

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_URI, "model")

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]

运行自定义训练任务

接下来，您通过调用`run`方法运行自定义作业，以启动训练作业，并传入以下参数：

- `args`：传递给训练脚本的命令行参数。
- `replica_count`：用于训练的计算实例数量（replica_count = 1 表示单节点训练）。
- `machine_type`：计算实例的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作器副本的加速器数量。
- `base_output_dir`：写入模型产物的 Cloud Storage 位置。
- `sync`：是否阻塞直到作业完成。

In [ ]:
if TRAIN_GPU:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        sync=True,
    )
else:
    job.run(
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=True,
    )

model_path_to_deploy = MODEL_DIR

## 加载已保存的模型

您的模型以 TensorFlow SavedModel 格式存储在 Cloud Storage 存储桶中。现在从 Cloud Storage 存储桶中加载它，然后您可以做一些事情，比如评估模型和进行预测。

要加载模型，您可以使用 TF.Keras `model.load_model()` 方法，将保存模型的 Cloud Storage 路径传递给它 -- 由 `MODEL_DIR` 指定。

In [ ]:
import tensorflow as tf

local_model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在找出模型有多好。

### 加载评估数据

从 `tf.keras.datasets` 加载 CIFAR10 测试（保留）数据，使用 `load_data()` 方法。这会将数据集作为包含两个元素的元组返回。第一个元素是训练数据，第二个是测试数据。每个元素也是包含两个元素的元组：图像数据和相应的标签。

你不需要训练数据，这就是为什么将其加载到`(_, _)`中。

在将数据传递给评估之前，你需要对其进行预处理：

`x_test`:
1. 通过将每个像素除以 255 来对像素数据进行归一化（重新缩放）。这会用介于 0 到 1 之间的 32 位浮点数替换每个单个字节整数像素。

`y_test`:<br/>
2. 标签目前是标量（稀疏）的。在 `trainer/task.py` 脚本的 `compile()` 步骤中已经注意到为稀疏标签进行了编译。

In [ ]:
import numpy as np
from tensorflow.keras.datasets import cifar10

(_, _), (x_test, y_test) = cifar10.load_data()
x_test = (x_test / 255.0).astype(np.float32)

print(x_test.shape, y_test.shape)

### 执行模型评估

现在评估一下自定义任务中的模型表现如何。

In [ ]:
local_model.evaluate(x_test, y_test)

### 为图像数据提供服务功能

要将图像传递给预测服务，您需要将压缩的（例如JPEG）图像字节编码为base 64-这样可以使内容在通过网络传输二进制数据时免受修改。由于这个部署模型期望将输入数据作为原始（未压缩）字节，您需要确保将base 64编码的数据转换回原始字节，然后作为输入传递给部署模型。

为了解决这个问题，定义一个服务函数（`serving_fn`）并将其附加到模型作为预处理步骤。添加一个`@tf.function`装饰器，这样服务功能就会与基础模型融合在一起（而不是在CPU上游处理）。

当您发送预测或解释请求时，请求的内容将被解码为Tensorflow字符串（`tf.string`），然后传递给服务函数（`serving_fn`）。服务函数会将`tf.string`预处理为原始（未压缩）的numpy字节（`preprocess_fn`），以满足模型的输入要求：
- `io.decode_jpeg`- 解压JPG图像，返回一个具有三个通道（RGB）的Tensorflow张量。
- `image.convert_image_dtype` - 将整数像素值更改为float 32。
- `image.resize` - 调整图像大小以匹配模型的输入形状。
- `resized / 255.0` - 将像素数据在0到1之间重新调整（归一化）。

在这一点上，数据可以传递给模型（`m_call`）。

#### XAI 签名

当服务函数与基础模型一起保存（`tf.saved_model.save`）时，您需要指定服务函数的输入层作为签名`serving_default`。

对于XAI图像模型，您需要从服务函数保存两个额外的签名：

- `xai_preprocess`：服务函数中的预处理功能。
- `xai_model`：调用模型的具体功能。

In [ ]:
CONCRETE_INPUT = "numpy_inputs"


def _preprocess(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(32, 32))
    rescale = tf.cast(resized / 255.0, tf.float32)
    return rescale


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_fn(bytes_inputs):
    decoded_images = tf.map_fn(
        _preprocess, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {
        CONCRETE_INPUT: decoded_images
    }  # User needs to make sure the key matches model's input


@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_fn(bytes_inputs):
    images = preprocess_fn(bytes_inputs)
    prob = m_call(**images)
    return prob


m_call = tf.function(local_model.call).get_concrete_function(
    [tf.TensorSpec(shape=[None, 32, 32, 3], dtype=tf.float32, name=CONCRETE_INPUT)]
)

tf.saved_model.save(
    local_model,
    model_path_to_deploy,
    signatures={
        "serving_default": serving_fn,
        # Required for XAI
        "xai_preprocess": preprocess_fn,
        "xai_model": m_call,
    },
)

获取服务功能签名

您可以将模型重新加载到内存中，通过查询每个层对应的签名来获取您模型的输入和输出层的签名。

在进行预测请求时，您需要将请求路由到服务功能而不是模型，因此您需要知道服务功能的输入层名称——这将在您进行预测请求时使用。

您还需要知道服务功能的输入和输出层的名称，以构建解释元数据——后面会进行讨论。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)
serving_output = list(loaded.signatures["serving_default"].structured_outputs.keys())[0]
print("Serving function output:", serving_output)

input_name = local_model.input.name
print("Model input name:", input_name)
output_name = local_model.output.name
print("Model output name:", output_name)

### 解释规范

要在进行预测时获得解释，您必须启用解释功能，并在将自定义模型上传到 Vertex `Model` 资源时设置相应的设置。这些设置被称为解释元数据，包括:

- `parameters`：这是用于解释模型的可解释性算法的规范。您可以选择以下选项之一：
  - Shapley - *注意*，不推荐用于图像数据-- 可能需要很长时间
  - XRAI
  - 综合梯度
- `metadata`：这是算法应用于您的自定义模型的规范。

#### 解释参数

让我们首先深入了解可解释性算法的设置。

#### Shapley

将结果分配给每个特征，并考虑特征的不同排列方式。该方法提供了对确切 Shapley 值的采样近似。

用例：
  - 对表格数据的分类和回归。

参数：

- `path_count`：算法处理的特征路径的数量。对于要求精确 Shapley 值的近似路径有 M! 条道路，其中 M 是特征的数量。对于 CIFAR10 数据集，这将是 784 (28*28)。

对于任何非常数量的特征，这将计算成本过高。您可以将通过特征的路径数量减少到 M * `path_count`。

#### 综合梯度

一种基于梯度的方法，用于高效地计算具有与 Shapley 值相同公理性质的特征归因。

用例：
  - 对表格数据的分类和回归。
  - 对图像数据的分类。

参数：

- `step_count`：这是用于近似剩余总和的步骤数。步骤越多，积分近似越准确。一般的经验法则是 50 步，但随着步数的增加，计算时间也会增加。

#### XRAI

基于综合梯度方法，XRAI 评估图像的重叠位置，创建一个显著性地图，突出显示图像的相关位置而不是像素。

用例：

  - 对图像数据的分类。

参数：

- `step_count`：这是用于近似剩余总和的步骤数。步骤越多，积分近似越准确。一般的经验法则是 50 步，但随着步数的增加，计算时间也会增加。

在下一个代码单元中，将变量 `XAI` 设置为您希望在自定义模型上使用的解释算法。

In [ ]:
XAI = "ig"  # [ shapley, ig, xrai ]

if XAI == "shapley":
    PARAMETERS = {"sampled_shapley_attribution": {"path_count": 10}}
elif XAI == "ig":
    PARAMETERS = {"integrated_gradients_attribution": {"step_count": 50}}
elif XAI == "xrai":
    PARAMETERS = {"xrai_attribution": {"step_count": 50}}

parameters = aiplatform.explain.ExplanationParameters(PARAMETERS)

### 说明元数据

让我们首先深入了解说明元数据，其中包括：

- `outputs`：输出中的标量值，用于解释的属性。例如，在分类的概率输出\[0.1, 0.2, 0.7\]中，我们想要解释0.7。考虑以下公式，其中输出为`y`，这就是我们想要解释的内容。

    y = f(x)
    
考虑以下公式，其中输出为`y`和`z`。由于您只能对一个标量值进行归因，您必须选择是要解释输出`y`还是`z`。假设在此示例中，模型是目标检测，y和z分别是边界框和对象分类。您会选择要解释哪个输出。

    y, z = f(x)

`outputs`的字典格式为：

    { "outputs": { "[your_display_name]":
                   "output_tensor_name": [layer]
                 }
    }

<blockquote>
 -  [your_display_name]：您为要解释的输出分配的可读名称。一个常见示例是“概率”。<br/>
 -  "output_tensor_name"：用于识别要解释的输出层的键/值字段。 <br/>
 -  [layer]：要解释的输出层。在单一任务模型中，例如表格回归器，它是模型中的最后（最顶层）层。
</blockquote>

- `inputs`：归因所需的特征 - 它们如何影响输出。考虑以下公式，其中`a`和`b`是特征。您必须选择要解释如何贡献的特征。假设此模型用于A/B测试，其中`a`是用于预测的数据项，`b`标识模型实例是A还是B。您会选择`a`（或某些子集）作为特征，而不选择`b`，因为它不会对预测产生影响。

    y = f(a,b)

`inputs`的最小字典格式为：

    { "inputs": { "[your_display_name]":
                  "input_tensor_name": [layer]
                 }
    }

<blockquote>
 -  [your_display_name]：您为要解释的输入分配的可读名称。一个常见示例是“特征”。<br/>
 -  "input_tensor_name"：用于识别特征归因的输入层的键/值字段。 <br/>
 -  [layer]：特征归因的输入层。在单一输入张量模型中，它是模型中的第一个（最底层）层。
</blockquote>

由于模型的输入为表格数据，您可以指定以下两个附加字段作为报告/可视化辅助：

<blockquote>
 - "modality": "image"：指示字段值为图像数据。

</blockquote>

由于模型的输入为图像，您可以指定以下附加字段作为报告/可视化辅助：

<blockquote>
 - "modality": "image"：指示字段值为图像数据。
</blockquote>

In [ ]:
random_baseline = np.random.rand(32, 32, 3)
input_baselines = [{"number_vaue": x} for x in random_baseline]

INPUT_METADATA = {"input_tensor_name": CONCRETE_INPUT, "modality": "image"}

OUTPUT_METADATA = {"output_tensor_name": serving_output}

input_metadata = aiplatform.explain.ExplanationMetadata.InputMetadata(INPUT_METADATA)
output_metadata = aiplatform.explain.ExplanationMetadata.OutputMetadata(OUTPUT_METADATA)

metadata = aiplatform.explain.ExplanationMetadata(
    inputs={"image": input_metadata}, outputs={"class": output_metadata}
)

## 上传模型

接下来，使用`Model.upload()`方法将您的模型上传到一个`Model`资源，需要以下参数：

- `display_name`: `Model`资源的可读名称。
- `artifact`: 存储训练模型工件的云存储位置。
- `serving_container_image_uri`: 用于提供服务的容器映像。
- `sync`: 是否异步或同步执行上传。
- `explanation_parameters`: 用于配置解释`Model`预测的参数。
- `explanation_metadata`: 描述解释`Model`输入和输出的元数据。

如果`upload()`方法以异步方式运行，则可以随后使用`wait()`方法阻塞直到完成。

In [ ]:
model = aiplatform.Model.upload(
    display_name="cifar10",
    artifact_uri=MODEL_DIR,
    serving_container_image_uri=DEPLOY_IMAGE,
    explanation_parameters=parameters,
    explanation_metadata=metadata,
    sync=False,
)

model.wait()

获取测试项目

使用数据集中测试（留置）部分的示例作为测试项目。

In [ ]:
test_image_1 = x_test[0]
test_label_1 = y_test[0]
test_image_2 = x_test[1]
test_label_2 = y_test[1]
print(test_image_1.shape)

### 准备请求内容
您将发送CIFAR10图像作为压缩的JPG图像，而不是未经压缩的原始字节：

- `cv2.imwrite`：使用openCV将未压缩的图像写入磁盘作为压缩的JPEG图像。
 - 从\[0,1)范围反向将图像数据反归一化为[0,255)。
 - 将32位浮点值转换为8位无符号整数。

In [ ]:
import cv2

cv2.imwrite("tmp1.jpg", (test_image_1 * 255).astype(np.uint8))
cv2.imwrite("tmp2.jpg", (test_image_2 * 255).astype(np.uint8))

### 复制测试项目

对于批量预测，请将测试项目复制到您的云存储桶中。

In [ ]:
! gsutil cp tmp1.jpg $BUCKET_URI/tmp1.jpg
! gsutil cp tmp2.jpg $BUCKET_URI/tmp2.jpg

test_item_1 = BUCKET_URI + "/tmp1.jpg"
test_item_2 = BUCKET_URI + "/tmp2.jpg"

### 制作批量输入文件

现在制作一个批量输入文件，然后将其存储到您的云存储桶中。 批量输入文件只能是JSONL格式。 对于JSONL文件，您为每个数据项（实例）的每一行制作一个字典条目。 该字典包含键/值对：

- `input_name`: 底层模型输入层的名称。
- `'b64'`: 表示内容已经进行base64编码。
- `content`: 将压缩后的JPG图像字节作为base64编码的字符串。

预测请求中的每个实例都是以下形式的字典条目：

                        {serving_input: {'b64': content}}
                        
为了将图像数据传递给预测服务，您需要将字节编码为base64。 这样在通过网络传输二进制数据时，可以确保内容不被修改。

- `tf.io.read_file`: 将压缩后的JPG图像读取为原始字节存入内存。
- `base64.b64encode`: 将原始字节编码为base64编码的字符串。

In [ ]:
import base64
import json

gcs_input_uri = BUCKET_URI + "/" + "test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    bytes = tf.io.read_file(test_item_1)
    b64str = base64.b64encode(bytes.numpy()).decode("utf-8")
    data = {serving_input: {"b64": b64str}}
    f.write(json.dumps(data) + "\n")
    bytes = tf.io.read_file(test_item_2)
    b64str = base64.b64encode(bytes.numpy()).decode("utf-8")
    data = {serving_input: {"b64": b64str}}
    f.write(json.dumps(data) + "\n")

### 发送批量预测请求

现在您的模型资源已经训练完成，您可以通过调用batch_predict()方法进行批量预测，使用以下参数：

- `job_display_name`：批量预测作业的人类可读名称。
- `gcs_source`：一个或多个批处理请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的云存储位置。
- `instances_format`：输入实例的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `predictions_format`：输出预测的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `machine_type`：用于训练的机器类型。
- `sync`：是否同步执行作业。如果为False，则作业将在并发Future中执行，并在Future完成时立即返回和同步任何下游对象。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

batch_predict_job = model.batch_predict(
    job_display_name="cifar10",
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="jsonl",
    model_parameters=None,
    machine_type=DEPLOY_COMPUTE,
    starting_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    generate_explanation=True,
    sync=False,
)

print(batch_predict_job)

### 等待批处理预测作业完成

接下来，等待批处理作业完成。或者，可以在`batch_predict（）`方法中将参数`sync`设置为`True`，以阻塞直到批处理预测作业完成。

In [ ]:
batch_predict_job.wait()

获取解释

接下来，从已完成的批量预测作业中获取解释结果。

结果被写入您在批量预测请求中指定的云存储输出桶中。您可以调用iter_outputs()方法来获取生成结果的每个云存储文件的列表。每个文件都以CSV格式包含一个或多个解释请求：

- CSV标头 + 预测标签
- CSV行 + 解释，每个预测请求一个

In [ ]:
import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

explanation_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("explanation"):
        explanation_results.append(blob.name)

tags = list()
for explanation_result in explanation_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{explanation_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            print(line)

# 清理

要清理此项目中使用的所有谷歌云资源，您可以[删除用于本教程的谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

将`delete_bucket`设置为**True**以删除云存储存储桶。

In [ ]:
delete_model = True
delete_batch_predict_job = True
delete_bucket = False
delete_application_directory = False

try:
    model.delete()
except Exception as e:
    print(e)

try:
    batch_predict_job.delete()
except Exception as e:
    print(e)

if delete_bucket:
    ! gsutil rm -r $BUCKET_URI

if delete_application_directory:
    ! rm -rf custom custom.tar.gz temp1.jpg temp2.jpg